# Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchsummary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings(action='ignore')

# Set Device

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Set Hyper-Parameters

In [3]:
notebookName = "DualEncoderRNN"
SEED = 20180724
batch_size = 1
learning_rate = 0.01
weight_decay = 1e-3
nepochs = 1000000
max_patience_count = 100

torch.manual_seed(SEED)

# Set Output Path

In [4]:

PATH = Path(f"./models/{notebookName}")
if os.path.isdir(PATH):
    dir_list = os.listdir(PATH)
    num_files = 0
    while True:
        if os.path.isfile(str(PATH / f"{num_files}")):
            num_files += 1
        else:
            break
else:
    os.mkdir(PATH)
    num_files = 0
num_files = 2

# Load Dataset

In [5]:
data_dir = Path("../input/google-smartphone-decimeter-challenge/")
df_train_default = pd.read_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))
df_test = pd.read_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))

In [6]:
def CustomTrainValidSplit(df:pd.DataFrame, valid_size):
    phones = df['phone'].unique()
    
    valid_num = int(len(phones) * valid_size)
    train_num = len(phones) - valid_num
    
    indexes = np.array(range(len(phones)))
    indexes = np.random.choice(indexes, len(indexes))
    
    df_train = []
    for phone in phones[indexes[:train_num]]:
        df_train.append(df[df['phone'] == phone])
    df_train = pd.concat(df_train)
    
    df_valid = []
    for phone in phones[indexes[train_num:-1]]:
        df_valid.append(df[df['phone'] == phone])
    df_valid = pd.concat(df_valid)
    
    return df_train.reset_index().drop(columns = 'index'), df_valid.reset_index().drop(columns = 'index')
    
df_train, df_valid = CustomTrainValidSplit(df_train_default, valid_size = 0.1)
print(df_train.shape, df_valid.shape)
del df_train_default

(118228, 148) (11364, 148)


In [23]:
class TimeSeriseDataset(torch.utils.data.Dataset):
    def __init__(self, df, phys_features, stat_features, labels, train = False):
        self.phys_features = phys_features
        self.stat_features = stat_features
        self.labels = labels
        self.train = train
        self.phones =df['phone'].unique()
        self._len = len(self.phones)
        
        self.df = df
        
    def __len__(self):
        return self._len
    
    def __getitem__(self, idx):
        phys_features = self.phys_features
        stat_features = self.stat_features
        labels = self.labels
        phone = self.phones[idx]
        df = self.df[self.df['phone']==phone]
        
        phys = torch.Tensor(df[phys_features].values)
        stat = torch.Tensor(df[stat_features].values)
        if self.train:
            label = torch.Tensor(df[labels].values)
        else:
            label = torch.Tensor([])
        return phys, stat, label

In [24]:
phys_features = [
    'latDeg', 
    'lngDeg', 
    'heightAboveWgs84EllipsoidM',
    'dlatDeg',
    'dlngDeg',
    'dheight',
    'UncalGyroXRadPerSec_Scaled',
    'UncalGyroYRadPerSec_Scaled',
    'UncalGyroZRadPerSec_Scaled',
    'DriftXRadPerSec_Scaled',
    'DriftYRadPerSec_Scaled',
    'DriftZRadPerSec_Scaled',
    'UncalAccelXMps2_Scaled',
    'UncalAccelYMps2_Scaled',
    'UncalAccelZMps2_Scaled',
    'BiasXMps2_Scaled',
    'BiasYMps2_Scaled',
    'BiasZMps2_Scaled',
    'UncalMagXMicroT_Scaled',
    'UncalMagYMicroT_Scaled',
    'UncalMagZMicroT_Scaled',
    'BiasXMicroT_Scaled',
    'BiasYMicroT_Scaled',
    'BiasZMicroT_Scaled',
    'yawDeg_Scaled',
    'rollDeg_Scaled',
    'pitchDeg_Scaled',
]
stat_features = [
    'GPS_L1', 
    'GPS_L5', 
    'GAL_E1', 
    'GAL_E5A', 
    'GLO_G1', 
    'BDS_B1I', 
    'BDS_B1C', 
    'BDS_B2A', 
    'QZS_J1', 
    'QZS_J5',
    'xSatPosM_Scaled',
    'ySatPosM_Scaled',
    'zSatPosM_Scaled',
    'xSatVelMps_Scaled',
    'ySatVelMps_Scaled',
    'zSatVelMps_Scaled',
    'State',
    'MultipathIndicator',
]

labels = [
    't_latDeg', 
    't_lngDeg', 
    't_heightAboveWgs84EllipsoidM',
#     'courseDegree',
#     'hDop',
#     'vDop',
#     'speedMps'
         ]

In [25]:
train_data = TimeSeriseDataset(df_train, 
                           phys_features, stat_features, labels,
                           train = True)
valid_data = TimeSeriseDataset(df_valid, 
                           phys_features, stat_features, labels,
                           train = True)
test_data = TimeSeriseDataset(df_test, 
                           phys_features, stat_features, labels,
                           train = False)
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_data, batch_size = batch_size, shuffle = False)

In [10]:

def torch_haversine(pred, true):

    lat1=pred[:,0] % 360
    lon1=pred[:,1] % 360
    lat2=true[:,0] % 360
    lon2=true[:,1] % 360

    lat1, lat2, lon1, lon2 = map(lambda x:x*np.pi/180, [lat1, lat2, lon1, lon2])

    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)

    a = torch.sin(dlat / 2.0)**2 + torch.cos(lat1) * torch.cos(lat2) * (torch.sin(dlon / 2.0)**2)
    c = 2 * torch.arcsin(a ** 0.5)

    dist = 6_367_000 * c

    return dist

def quantile_mean(dist):
    return (torch.quantile(dist, 0.5) + torch.quantile(dist, 0.95))/2
    
def gps_loss(predict:torch.Tensor, target:torch.Tensor):
#     dist = torch_haversine(predict, target)
#     loss = dist.mean()

    loss = nn.SmoothL1Loss()(predict, target)

    return loss

def gps_score(predict:torch.Tensor, target:torch.Tensor):
    dist = torch_haversine(predict, target)

    score = quantile_mean(dist)

    return score

def calc_loss_and_score(predicts, grounds):
    dist = []
    for pred, ground in zip(predicts, grounds):
        pred = pred.squeeze(0)
        ground = ground.squeeze(0)
        dist.append(torch_haversine(pred, ground))
    dist = torch.cat(dist, axis = 0)
    return dist.mean(), quantile_mean(dist)

In [11]:
class Encoder(nn.Module):
    def __init__(self, num_features, num_hiddens, num_layers, dropout = 0.1, device_ = 'cpu'):
        super().__init__()
        self.device = device_
        
        self.gru = nn.GRU(num_features, num_hiddens, num_layers, batch_first = True, dropout = dropout, bidirectional = True)
    def forward(self, x):
        x, h = self.gru(x)
        h = h.to(device)
        return x, h

In [12]:
class Decoder(nn.Module):
    def __init__(self, num_features, num_hiddens, num_layers, dropout = 0.1, device_ = 'cpu'):
        super().__init__()
        self.device = device_
        
        self.gru = nn.GRU(num_features, num_hiddens, num_layers, batch_first = True, dropout = dropout, bidirectional = True)
    def forward(self, x, h0):
        x, h = self.gru(x, h0)
        h = h.to(device)
        return x, h

In [13]:
class BaseModel(nn.Module):
    def __init__(self, num_state, num_status, num_outputs, device_ = 'cpu'):
        super().__init__()
        self.device = device_
        self.dropout03 = nn.Dropout(0.3)
        
        self.embd1 = nn.Conv1d(num_status, 64, 1)
        self.embd2 = nn.Conv1d(64, 128, 1)
        self.embd3 = nn.Conv1d(128, 128, 1)
        
        self.phys_encoder = Encoder(num_state, 32, 1, 0.6, device_)
        self.stat_encoder = Encoder(128, 32, 1, 0.6, device_)
        self.decoder = Decoder(num_state, 64, 1, 0.3, device_)
        
        self.conv1 = nn.Conv1d(128, 64, 1)
        self.conv2 = nn.Conv1d(64, 32, 1)
        self.conv3 = nn.Conv1d(32, 3, 1)
        
        self.batch1 = nn.BatchNorm1d(64)
        self.batch2 = nn.BatchNorm1d(32)
        self.batch3 = nn.BatchNorm1d(3)
        
        pass
    def forward(self, phys, stat):
        stat = self.dropout03(stat)
        phys = self.dropout03(phys)
        
        stat = stat.transpose(1,2)
        stat = self.embd1(stat)
        stat = F.relu(stat)
        stat = self.embd2(stat)
        stat = F.relu(stat)
        stat = self.embd3(stat)
        stat = stat.transpose(1,2)
        
        phys_x, phys_h = self.phys_encoder(phys)
        stat_x, stat_h = self.stat_encoder(stat)

        h0 = torch.cat([phys_h, stat_h], axis = 2)
        
        out, _ = self.decoder(phys, h0)
        
        out = out.transpose(1,2)
        out = self.conv1(out)
        out = F.tanh(out)
        out = self.batch1(out)
        out = self.conv2(out)
        out = F.tanh(out)
        out = self.batch2(out)
        out = self.conv3(out)
        out = out.transpose(1,2)
        
        out = out + phys_x[:,:,:3]
        return out

In [14]:
model = BaseModel(len(phys_features), len(stat_features), len(labels), device)
model.to(device)
# model.load_state_dict(torch.load("./models/Baseline3/model-4.pth"))

optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay = weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                mode = 'min',
                                                factor = 0.1,
                                                patience = 5,
                                                verbose = True)

In [15]:
def train(epoch, progress_log):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    for phys, stat, label in progress_log:
        
        phys = phys.to(device)
        stat = stat.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        pred = model(phys, stat)  # 데이터를 입력하고 출력을 계산
        
        loss = gps_loss(pred, label)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        score = gps_score(pred, label)  # 출력과 훈련 데이터 정답 간의 오차를 계산

        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        predict.append(pred)
        ground.append(label)
            
    loss, score = calc_loss_and_score(predict, ground)
    
    del predict, ground
    return loss, score

In [16]:
def valid(epoch, progress_log):
    model.eval()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    with torch.no_grad():
        for phys, stat, label in progress_log:

            phys = phys.to(device)
            stat = stat.to(device)
            label = label.to(device)

            pred = model(phys, stat)  # 데이터를 입력하고 출력을 계산

            predict.append(pred)
            ground.append(label)
            
    loss, score = calc_loss_and_score(predict, ground)
    
    del predict, ground
    return loss, score

In [17]:
def test(epoch, progress_log):
    model.eval()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    with torch.no_grad():
        for phys, stat, _ in progress_log:

            phys = phys.to(device)
            stat = stat.to(device)

            pred = model(phys, stat)  # 데이터를 입력하고 출력을 계산

            predict.append(pred.cpu())
    
    return predict

In [18]:
train_loss_list = []
train_score_list = []
valid_loss_list = []
valid_score_list = []

patience_count = 0
min_valid_score = np.inf
checkpoint_name = ""

if not os.path.isdir(f"./models/{notebookName}/model-{num_files}_checkpoint/"):
    os.mkdir(f"./models/{notebookName}/model-{num_files}_checkpoint/")
    
prog_epoch = tqdm(range(0, nepochs), position = 0, desc = 'EPOCH')
for epoch in prog_epoch:
    print( "-------------------------------------------------------")
    print(f"|EPOCH: {epoch+1}/{nepochs}")
    prog_train = tqdm(train_loader, desc = 'TRAIN', leave = False)
    prog_valid = tqdm(valid_loader, desc = 'VALID', leave = False)
    
    train_loss, train_score = train(epoch, prog_train)
    valid_loss, valid_score = valid(epoch, prog_valid)
    
    scheduler.step(valid_score)
    
    if min_valid_score > valid_score:
        print("|IS Best!")
        min_valid_score = valid_score
        checkpoint_name = f"./models/{notebookName}/model-{num_files}_checkpoint/model-{epoch}-{valid_score}.pth"
        torch.save(model.state_dict(), checkpoint_name)
    else:
        patience_count+=1
        if(patience_count > max_patience_count):
            break
        
    train_loss_list.append(train_loss)
    train_score_list.append(train_score)
    valid_loss_list.append(valid_loss)
    valid_score_list.append(valid_score)
    
    print(f"|TRAIN: loss={train_loss:.6f},  score={train_score:.6f}|")
    print(f"|VALID: loss={valid_loss:.6f},  score={valid_score:.6f}|")
    
    
history = dict()
history['train_loss'] = train_loss_list
history['train_score'] = train_score_list
history['valid_loss'] = valid_loss_list
history['valid_score'] = valid_score_list

EPOCH:   0%|          | 0/1000000 [00:00<?, ?it/s]

-------------------------------------------------------
|EPOCH: 1/1000000


TRAIN:   0%|          | 0/45 [00:00<?, ?it/s]

VALID:   0%|          | 0/6 [00:00<?, ?it/s]

|IS Best!
|TRAIN: loss=12109770.000000,  score=12517504.000000|
|VALID: loss=11010718.000000,  score=11014408.000000|
-------------------------------------------------------
|EPOCH: 2/1000000


...

TRAIN:   0%|          | 0/45 [00:00<?, ?it/s]

VALID:   0%|          | 0/6 [00:00<?, ?it/s]

|TRAIN: loss=18378.664062,  score=32696.128906|
|VALID: loss=21483.330078,  score=30328.021484|
-------------------------------------------------------
|EPOCH: 111/1000000


TRAIN:   0%|          | 0/45 [00:00<?, ?it/s]

VALID:   0%|          | 0/6 [00:00<?, ?it/s]

In [19]:
# Load submission sample
submission = pd.read_csv(str(data_dir / "sample_submission.csv"))
print(submission.shape)
submission.head()

(91486, 4)


,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.904611,-86.481078
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.904611,-86.481078
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.904611,-86.481078
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.904611,-86.481078
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.904611,-86.481078


In [20]:
model.load_state_dict(torch.load(checkpoint_name))
torch.save(model.state_dict(), f"./models/{notebookName}/model-{min_valid_score}.pth")

In [26]:
predict = test(0, tqdm(test_loader))

  0%|          | 0/48 [00:00<?, ?it/s]

In [27]:
predict = torch.cat(predict, axis = 1).squeeze(0)
submission['latDeg'] = predict[:,0]
submission['lngDeg'] = predict[:,1]

In [28]:
submission.to_csv(f"./models/{notebookName}/result-{num_files}-{min_valid_score}.csv", index = False)
pd.DataFrame([]).to_csv(PATH / f"{num_files}")